# Kryptologie Lab - Übung 05

In [2]:
from tqdm.notebook import tqdm

In [3]:
sbox = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]
perm = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

In [4]:
def sbox_block(value, inverse=False):
    if inverse:
        return sbox.index(value)
    else: 
        return sbox[value]

def executeSbox(value,inverse=False):
    block0 = sbox_block(value & 0xF, inverse)
    block1 = sbox_block((value & 0xF0) >> 4, inverse)
    block2 = sbox_block((value & 0xF00) >> 8, inverse)
    block3 = sbox_block((value & 0xF000) >> 12, inverse)
    return (block3 << 12) | (block2 << 8) | (block1 << 4) | block0

def permutation(value,inverse=False):
    output = 0
    for i in range(1, 16):
        bit = value & 0x1
        output = output | (bit << (perm[i] - 1))
        value = value >> 1
    return output

def addRoundKey(value, key):
    return value ^ key

In [5]:
def encrypt(value, key):
    for i in range(3):
        value = addRoundKey(value, key)
        value = executeSbox(value)
        value = permutation(value)
    value = addRoundKey(value, key)
    value = executeSbox(value)
    value = addRoundKey(value, key)
    return value

In [6]:
for i in range(0, 16):
    encrypted = sbox_block(i)
    decrypted = sbox_block(encrypted, inverse=True)
    assert i == decrypted

value = 0xABCD
encrypted = executeSbox(value)
decrypted = executeSbox(encrypted, inverse=True)
assert value == decrypted

encrypt(0xABCD, 0xFABE)

50256

## Lineare Kryptonanalyse

In [7]:
import random

In [8]:
def getBit(val, position):
    return (val >> position) & 0x1

def expandKey(subkey):
    return ((subkey & 0xF) << 4) ^ ((subkey & 0xF0) << 8)

def getProbabilityOfKeys(pairs):
    possibleKeys = dict()
    # iterate possible keys
    for subkey in range(0xFF+1):
        key = expandKey(subkey)
        hits = 0
        # iterate text-cipher pairs
        for pair in pairs:
            x = pair[0]
            y = pair[1]
            v4 = addRoundKey(y, key)
            u4 = executeSbox(v4, inverse=False)
            approx = getBit(x,4) ^ getBit(x,6) ^ getBit(x,7) ^ getBit(u4,5) ^ getBit(u4,7) ^ getBit(u4,13) ^ getBit(u4,15)
            if approx == 0:
                hits += 1
        possibleKeys[subkey] = hits/len(pairs)
    return possibleKeys

def getBestSubKeyOrder(probabilities):
    for data in probabilities.items():
        probabilities.update({data[0]: abs(data[1] - 0.5)})
    return sorted(probabilities.items(), key=lambda item: item[1], reverse=True)

In [9]:
key = 0xFFAE
amount = 12000

M = []
for i in tqdm(range(amount)):
    input = random.getrandbits(16)
    M.append((input, encrypt(input, key)))

In [10]:
probabilities = getProbabilityOfKeys(M)
subkeysInOrderWithProbability = getBestSubKeyOrder(probabilities)
print(subkeysInOrderWithProbability)
subkeysInOrder = [entry[0] for entry in subkeysInOrderWithProbability]
print(subkeysInOrder)

[(78, 0.015666666666666662), (225, 0.014333333333333309), (96, 0.013499999999999956), (229, 0.013416666666666688), (227, 0.012333333333333307), (226, 0.011916666666666687), (74, 0.011750000000000038), (101, 0.011499999999999955), (95, 0.011333333333333306), (110, 0.011249999999999982), (107, 0.011083333333333334), (124, 0.011083333333333334), (79, 0.010750000000000037), (125, 0.010666666666666658), (215, 0.010499999999999954), (115, 0.010249999999999981), (65, 0.010000000000000009), (224, 0.009749999999999981), (61, 0.009333333333333305), (109, 0.009333333333333305), (133, 0.009166666666666656), (191, 0.009083333333333332), (238, 0.009000000000000008), (83, 0.008916666666666684), (163, 0.008916666666666684), (45, 0.00883333333333336), (85, 0.00883333333333336), (230, 0.00883333333333336), (178, 0.00874999999999998), (187, 0.008500000000000008), (189, 0.008416666666666683), (126, 0.008416666666666628), (62, 0.00824999999999998), (140, 0.00824999999999998), (25, 0.008166666666666655), (5

## Brutforce in better order

In [13]:
countKeyTries = 0
cracked = False
for key in tqdm(subkeysInOrder):
    countKeyTries += 1
    for brutforceSubkey in range(0xFF+1):
        expandedKey = expandKey(key)
        testKey = (brutforceSubkey & 0xF) ^ ((brutforceSubkey & 0xF0) << 4) ^ expandedKey
        cipher = encrypt(M[0][0], testKey)
        if cipher == M[0][1]:
            correct = 0
            for m in M:
                cipher = encrypt(m[0], testKey)
                if cipher == m[1]:
                    correct += 1
            if correct == len(M):
                print("Key: ", hex(testKey))
                print("Cracked after ", countKeyTries, " tries")
                cracked = True
                break
    if cracked:
        break       

Key:  0xffae
Cracked after  81  tries

